# Models Testing on Heston data


In [1]:
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [2]:
# Set seeds
torch.manual_seed(0)
np.random.seed(0)

In [3]:
synthetic_calls_path = '../data/heston_mc_synthetic_calls_tot.csv'
synthetic_puts_path = '../data/heston_mc_synthetic_puts_tot.csv'

In [4]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')
    
    return df

In [5]:
synthetic_calls = pd.read_csv(synthetic_calls_path, index_col=0)
synthetic_puts = pd.read_csv(synthetic_puts_path, index_col=0)

synthetic_calls = reduce_mem_usage(synthetic_calls)
synthetic_puts = reduce_mem_usage(synthetic_puts)

In [6]:
synthetic_options = pd.concat([synthetic_calls, synthetic_puts], axis=0)
synthetic_options = shuffle(synthetic_options, random_state=0)
synthetic_options = synthetic_options.reset_index()
synthetic_options = synthetic_options.drop('index', axis=1)

In [7]:
synthetic_options

,Price,Strike,Type,Kappa,Rho,Theta,Xi,V_0,Interest Rate,Time to Expiration,Option Price
0,100,143.0,C,1.728516,0.158936,0.158936,0.158936,0.158936,0.058685,0.158936,0.000000
1,100,68.0,P,1.207031,0.547852,0.547852,0.547852,0.547852,0.073486,0.547852,0.957520
2,100,127.0,P,0.997070,0.672363,0.672363,0.672363,0.672363,0.014740,0.672363,26.546875
3,100,140.0,P,1.411133,0.494629,0.494629,0.494629,0.494629,0.052063,0.494629,38.750000
4,100,60.0,C,0.271729,0.102478,0.102478,0.102478,0.102478,0.035553,0.102478,38.593750
...,...,...,...,...,...,...,...,...,...,...,...
807995,100,71.0,C,0.553223,0.829590,0.829590,0.829590,0.829590,0.076904,0.829590,27.875000
807996,100,57.0,C,1.411133,0.825684,0.825684,0.825684,0.825684,0.084656,0.825684,42.437500
807997,100,135.0,C,1.419922,1.062500,1.062500,1.062500,1.062500,0.073792,1.062500,2.345703
807998,100,64.0,P,1.577148,1.038086,1.038086,1.038086,1.038086,0.091187,1.038086,0.202026


# Preprocessing

In [8]:
synthetic_options = pd.get_dummies(synthetic_options, prefix='', prefix_sep='')

In [9]:
input_sc = StandardScaler()
output_sc = StandardScaler()
input_data = input_sc.fit_transform(synthetic_options.drop('Option Price', axis=1))
output_data = output_sc.fit_transform(synthetic_options['Option Price'].values.reshape(-1, 1))

train_size = 0.8
val_size = 0.1

last_train_idx = int(np.round(len(input_data) * train_size))
last_val_idx = last_train_idx + int(np.round(len(input_data) * val_size))

X_train = input_data[0:last_train_idx]
X_val = input_data[last_train_idx:last_val_idx]
X_test = input_data[last_val_idx:]

y_train = output_data[0:last_train_idx]
y_val = output_data[last_train_idx:last_val_idx]
y_test = output_data[last_val_idx:]

In [10]:
X_train = Variable(torch.Tensor(X_train))
X_val = Variable(torch.Tensor(X_val))
X_test = Variable(torch.Tensor(X_test))

y_train = Variable(torch.Tensor(y_train))
y_val = Variable(torch.Tensor(y_val))
y_test = Variable(torch.Tensor(y_test))

# Model

In [11]:
CUDA = torch.cuda.is_available()
device = 'cuda:0' if CUDA else 'cpu'

In [12]:
class ResBlock(nn.Module):

  def __init__(self, module):
    super(ResBlock, self).__init__()
    self.module = module

  def forward(self, x):
    return self.module(x) + x

In [13]:
class HiddenLayer(nn.Module):

  def __init__(self, layer_size, act_fn):
      super(HiddenLayer, self).__init__()
      
      if act_fn == 'ReLU':
        self.layer = nn.Sequential(
          nn.Linear(layer_size, layer_size),
          nn.ReLU())
      elif act_fn == 'LeakyReLU':
        self.layer = nn.Sequential(
          nn.Linear(layer_size, layer_size),
          nn.LeakyReLU())
      elif act_fn == 'ELU':
        self.layer = nn.Sequential(
          nn.Linear(layer_size, layer_size),
          nn.ELU())
    
  def forward(self, x):
    return self.layer(x)

In [14]:
class Net(nn.Module):

  def __init__(self, input_size, output_size, hidden_size, num_layers, act_fn):
    super(Net, self).__init__()
    self.input_size = input_size
    self.output_size = output_size
    self.hidden_size = hidden_size

    if act_fn == 'ReLU':
      self.initial_layer = nn.Sequential(
          nn.Linear(self.input_size, self.hidden_size),
          nn.ReLU())
    elif act_fn == 'LeakyReLU':
      self.initial_layer = nn.Sequential(
          nn.Linear(self.input_size, self.hidden_size),
          nn.LeakyReLU())
    elif act_fn == 'ELU':
      self.initial_layer = nn.Sequential(
          nn.Linear(self.input_size, self.hidden_size),
          nn.ELU())

    self.hidden_layers_list = []

    for i in range(num_layers // 2):
      self.hidden_layers_list.append(
          ResBlock(
            nn.Sequential(
                HiddenLayer(self.hidden_size, act_fn),
                HiddenLayer(self.hidden_size, act_fn)
            )
        )
      )

    self.hidden_layers = nn.Sequential(*self.hidden_layers_list)

    self.net = nn.Sequential(
        self.initial_layer,
        self.hidden_layers,
        nn.Linear(self.hidden_size, self.output_size)
    )
  
  def forward(self, x):
    return self.net(x)

In [15]:
def init_weights(m, init_m: str):

  @torch.no_grad()
  def init_uniform(m):
    if isinstance(m, nn.Linear):
      torch.nn.init.uniform_(m.weight)
      m.bias.data.fill_(0.01)

  @torch.no_grad()
  def init_normal(m):
    if isinstance(m, nn.Linear):
      torch.nn.init.normal_(m.weight)
      m.bias.data.fill_(0.01)

  @torch.no_grad()
  def init_xuniform(m):
    if isinstance(m, nn.Linear):
      torch.nn.init.xavier_uniform_(m.weight)
      m.bias.data.fill_(0.01)

  @torch.no_grad()
  def init_xnormal(m):
    if isinstance(m, nn.Linear):
      torch.nn.init.xavier_normal_(m.weight)
      m.bias.data.fill_(0.01)

  if init_m == 'uniform':
    m.apply(init_uniform)
  elif init_m == 'normal':
    m.apply(init_normal)
  elif init_m == 'xaiver uniform':
    m.apply(init_xuniform)
  elif init_m == 'xavier normal':
    m.apply(init_xnormal)

# Best models from cross validation

In [16]:
best_models = [
               {'n_hidden': 400, 'n_layers': 8, 'act_fun': 'LeakyReLU', 'init_method': 'xavier uniform'},
               {'n_hidden': 400, 'n_layers': 4, 'act_fun': 'ReLU', 'init_method': 'xavier uniform'},
               {'n_hidden': 400, 'n_layers': 4, 'act_fun': 'LeakyReLU', 'init_method': 'xavier uniform'},
               {'n_hidden': 400, 'n_layers': 6, 'act_fun': 'LeakyReLU', 'init_method': 'xavier uniform'},
               {'n_hidden': 600, 'n_layers': 4, 'act_fun': 'LeakyReLU', 'init_method': 'xavier uniform'}
]

# Training

In [17]:
input_size = 11
output_size = 1
batch_size = 1024
epochs = 125
lr = 1e-4

loss_fn = nn.MSELoss()

In [18]:
class OptDataset(Dataset):

  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]

  def __len__(self):
    return len(self.X)

In [19]:
def evaluate(model, loss_fn, X, y):
  model.eval()
  with torch.no_grad():
    out = model(X.to(device))
    loss = loss_fn(out, y.to(device))
    return loss.item()

In [20]:
def mape_loss(y_pred, y):
  return sum([np.abs(y_i - hy_i) / y_i for y_i, hy_i in zip(y_pred, y)]) / len(y_pred)

In [21]:
def get_mape_loss(model, X, y):
  model.eval()
  with torch.no_grad():
    out = model(X.to(device))
    loss = mape_loss(
        out.squeeze().cpu().detach().numpy(), 
        y.squeeze().cpu().detach().numpy())
    return loss

In [22]:
def test_models(
  models_dict,
  epochs,
  batch_size,
  X_train,
  y_train,
  X_val,
  y_val,
  X_test,
  y_test,
  loss_fn
):
  testing_results = pd.DataFrame(columns=
                               ['hidden_size',
                                'n_layers',
                                'act_fun',
                                'init_methods',
                                'mean_val_result',
                                'std_val_result',
                                'test_mse',
                                'test_mae',
                                'test_rmse',
                                'test_mape'])

  for model_dict in models_dict:
    model = Net(input_size,
                output_size, 
                model_dict['n_hidden'], 
                model_dict['n_layers'], 
                model_dict['act_fun']).to(device)
    init_weights(model, model_dict['init_method'])
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    validation_losses = []
    test_res = {
        'hidden_size': model_dict['n_hidden'],
        'n_layers': model_dict['n_layers'],
        'act_fun': model_dict['act_fun'],
        'init_methods': model_dict['init_method']
    }

    print('Model: ', test_res)

    for epoch in range(epochs):
      model.train()
      total_loss = 0
      start_time = time.time()
      i = 0

      for batch, batch_labels in DataLoader(OptDataset(X_train, y_train), batch_size):
        out = model(batch.to(device))
        optimizer.zero_grad()

        loss = loss_fn(out, batch_labels.to(device))
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        if i > 0 and i % 25 == 0:
          avg_loss = total_loss / 50
          elapsed = time.time() - start_time
          print('| Epoch {:3d} | {:5d}/{:5d} batches | lr {:2.5f} | ms/batch {:5.2f} | '
                  'loss {:5.8f}'.format(
              epoch, i, len(X_train) // batch_size+1, lr, elapsed * 1000 / 50,
              avg_loss))
          start_time = time.time()
          total_loss = 0
        
        i += 1

      validation_losses.append(evaluate(model, loss_fn, X_val, y_val))
    
    mse_test = evaluate(model, loss_fn, X_test, y_test)
    mae_loss = nn.L1Loss()
    mae_test = evaluate(model, mae_loss, X_test, y_test)
    mape_test = get_mape_loss(model, X_test, y_test)
    validation_losses = np.array(validation_losses)
    test_res['mean_val_result'] = validation_losses.mean()
    test_res['std_val_result'] = validation_losses.std()
    test_res['test_mse'] = mse_test
    test_res['test_mae'] = mae_test
    test_res['test_rmse'] = np.sqrt(mse_test)
    test_res['test_mape'] = mape_test

    testing_results = testing_results.append(test_res, ignore_index=True)

  return testing_results

In [23]:
testing_results = test_models(best_models,
            epochs,
            batch_size,
             X_train,
            y_train,
            X_val,
            y_val,
            X_test,
            y_test,
            loss_fn)

Model:  {'hidden_size': 400, 'n_layers': 8, 'act_fun': 'LeakyReLU', 'init_methods': 'xavier uniform'}
| Epoch   0 |    25/  632 batches | lr 0.00010 | ms/batch 12.81 | loss 0.25558726
| Epoch   0 |    50/  632 batches | lr 0.00010 | ms/batch  8.86 | loss 0.02357464
| Epoch   0 |    75/  632 batches | lr 0.00010 | ms/batch 10.18 | loss 0.00670563
| Epoch   0 |   100/  632 batches | lr 0.00010 | ms/batch 10.22 | loss 0.00426822
| Epoch   0 |   125/  632 batches | lr 0.00010 | ms/batch 10.23 | loss 0.00337733
| Epoch   0 |   150/  632 batches | lr 0.00010 | ms/batch  8.68 | loss 0.00292318
| Epoch   0 |   175/  632 batches | lr 0.00010 | ms/batch 10.30 | loss 0.00275504
| Epoch   0 |   200/  632 batches | lr 0.00010 | ms/batch 10.21 | loss 0.00267560
| Epoch   0 |   225/  632 batches | lr 0.00010 | ms/batch 10.19 | loss 0.00255837
| Epoch   0 |   250/  632 batches | lr 0.00010 | ms/batch  8.70 | loss 0.00264991
| Epoch   0 |   275/  632 batches | lr 0.00010 | ms/batch 10.20 | loss 0.00251

| Epoch   3 |   625/  632 batches | lr 0.00010 | ms/batch 10.20 | loss 0.00231830
| Epoch   4 |    25/  632 batches | lr 0.00010 | ms/batch 10.58 | loss 0.00238695
| Epoch   4 |    50/  632 batches | lr 0.00010 | ms/batch 10.44 | loss 0.00236620
| Epoch   4 |    75/  632 batches | lr 0.00010 | ms/batch  8.62 | loss 0.00240161
| Epoch   4 |   100/  632 batches | lr 0.00010 | ms/batch 10.60 | loss 0.00238348
| Epoch   4 |   125/  632 batches | lr 0.00010 | ms/batch 10.04 | loss 0.00236274
| Epoch   4 |   150/  632 batches | lr 0.00010 | ms/batch 10.10 | loss 0.00236627
| Epoch   4 |   175/  632 batches | lr 0.00010 | ms/batch  8.88 | loss 0.00242295
| Epoch   4 |   200/  632 batches | lr 0.00010 | ms/batch 10.26 | loss 0.00233900
| Epoch   4 |   225/  632 batches | lr 0.00010 | ms/batch 10.26 | loss 0.00227230
| Epoch   4 |   250/  632 batches | lr 0.00010 | ms/batch 10.52 | loss 0.00239753
| Epoch   4 |   275/  632 batches | lr 0.00010 | ms/batch 10.09 | loss 0.00228825
| Epoch   4 |   

| Epoch   7 |   625/  632 batches | lr 0.00010 | ms/batch 10.32 | loss 0.00229823
| Epoch   8 |    25/  632 batches | lr 0.00010 | ms/batch 10.66 | loss 0.00235128
| Epoch   8 |    50/  632 batches | lr 0.00010 | ms/batch  8.73 | loss 0.00231314
| Epoch   8 |    75/  632 batches | lr 0.00010 | ms/batch 10.49 | loss 0.00230481
| Epoch   8 |   100/  632 batches | lr 0.00010 | ms/batch 10.20 | loss 0.00235875
| Epoch   8 |   125/  632 batches | lr 0.00010 | ms/batch 12.19 | loss 0.00230100
| Epoch   8 |   150/  632 batches | lr 0.00010 | ms/batch  9.23 | loss 0.00238436
| Epoch   8 |   175/  632 batches | lr 0.00010 | ms/batch 10.59 | loss 0.00235504
| Epoch   8 |   200/  632 batches | lr 0.00010 | ms/batch 10.44 | loss 0.00228544
| Epoch   8 |   225/  632 batches | lr 0.00010 | ms/batch 10.23 | loss 0.00224537
| Epoch   8 |   250/  632 batches | lr 0.00010 | ms/batch 10.74 | loss 0.00235114
| Epoch   8 |   275/  632 batches | lr 0.00010 | ms/batch  8.93 | loss 0.00224724
| Epoch   8 |   

| Epoch  11 |   625/  632 batches | lr 0.00010 | ms/batch 10.33 | loss 0.00228982
| Epoch  12 |    25/  632 batches | lr 0.00010 | ms/batch  8.84 | loss 0.00230187
| Epoch  12 |    50/  632 batches | lr 0.00010 | ms/batch 10.29 | loss 0.00226631
| Epoch  12 |    75/  632 batches | lr 0.00010 | ms/batch 10.12 | loss 0.00228194
| Epoch  12 |   100/  632 batches | lr 0.00010 | ms/batch  8.61 | loss 0.00228851
| Epoch  12 |   125/  632 batches | lr 0.00010 | ms/batch 10.15 | loss 0.00228862
| Epoch  12 |   150/  632 batches | lr 0.00010 | ms/batch 10.30 | loss 0.00234553
| Epoch  12 |   175/  632 batches | lr 0.00010 | ms/batch 10.21 | loss 0.00223390
| Epoch  12 |   200/  632 batches | lr 0.00010 | ms/batch  8.59 | loss 0.00224016
| Epoch  12 |   225/  632 batches | lr 0.00010 | ms/batch 10.13 | loss 0.00222175
| Epoch  12 |   250/  632 batches | lr 0.00010 | ms/batch 10.21 | loss 0.00231531
| Epoch  12 |   275/  632 batches | lr 0.00010 | ms/batch 10.14 | loss 0.00220924
| Epoch  12 |   

| Epoch  15 |   625/  632 batches | lr 0.00010 | ms/batch 10.15 | loss 0.00227865
| Epoch  16 |    25/  632 batches | lr 0.00010 | ms/batch 10.46 | loss 0.00226291
| Epoch  16 |    50/  632 batches | lr 0.00010 | ms/batch 10.11 | loss 0.00225078
| Epoch  16 |    75/  632 batches | lr 0.00010 | ms/batch  8.61 | loss 0.00225262
| Epoch  16 |   100/  632 batches | lr 0.00010 | ms/batch 10.18 | loss 0.00223547
| Epoch  16 |   125/  632 batches | lr 0.00010 | ms/batch 10.26 | loss 0.00227339
| Epoch  16 |   150/  632 batches | lr 0.00010 | ms/batch 10.28 | loss 0.00227308
| Epoch  16 |   175/  632 batches | lr 0.00010 | ms/batch  8.67 | loss 0.00217889
| Epoch  16 |   200/  632 batches | lr 0.00010 | ms/batch 10.27 | loss 0.00220518
| Epoch  16 |   225/  632 batches | lr 0.00010 | ms/batch 10.26 | loss 0.00219236
| Epoch  16 |   250/  632 batches | lr 0.00010 | ms/batch 10.26 | loss 0.00227087
| Epoch  16 |   275/  632 batches | lr 0.00010 | ms/batch  8.63 | loss 0.00217299
| Epoch  16 |   

...

| Epoch 107 |   525/  632 batches | lr 0.00010 | ms/batch  9.40 | loss 0.00089970
| Epoch 107 |   550/  632 batches | lr 0.00010 | ms/batch  9.47 | loss 0.00103115
| Epoch 107 |   575/  632 batches | lr 0.00010 | ms/batch  9.29 | loss 0.00094297
| Epoch 107 |   600/  632 batches | lr 0.00010 | ms/batch  7.78 | loss 0.00090165
| Epoch 107 |   625/  632 batches | lr 0.00010 | ms/batch  9.34 | loss 0.00090282
| Epoch 108 |    25/  632 batches | lr 0.00010 | ms/batch  9.67 | loss 0.00097781
| Epoch 108 |    50/  632 batches | lr 0.00010 | ms/batch  9.25 | loss 0.00092744
| Epoch 108 |    75/  632 batches | lr 0.00010 | ms/batch  7.68 | loss 0.00087385
| Epoch 108 |   100/  632 batches | lr 0.00010 | ms/batch  9.23 | loss 0.00087811
| Epoch 108 |   125/  632 batches | lr 0.00010 | ms/batch  9.36 | loss 0.00094878
| Epoch 108 |   150/  632 batches | lr 0.00010 | ms/batch  9.33 | loss 0.00095392
| Epoch 108 |   175/  632 batches | lr 0.00010 | ms/batch  7.65 | loss 0.00089687
| Epoch 108 |   

| Epoch 111 |   525/  632 batches | lr 0.00010 | ms/batch  9.35 | loss 0.00086913
| Epoch 111 |   550/  632 batches | lr 0.00010 | ms/batch  9.31 | loss 0.00099814
| Epoch 111 |   575/  632 batches | lr 0.00010 | ms/batch  7.81 | loss 0.00090352
| Epoch 111 |   600/  632 batches | lr 0.00010 | ms/batch  9.49 | loss 0.00085714
| Epoch 111 |   625/  632 batches | lr 0.00010 | ms/batch  9.43 | loss 0.00086026
| Epoch 112 |    25/  632 batches | lr 0.00010 | ms/batch 10.02 | loss 0.00093358
| Epoch 112 |    50/  632 batches | lr 0.00010 | ms/batch  8.09 | loss 0.00088208
| Epoch 112 |    75/  632 batches | lr 0.00010 | ms/batch  9.40 | loss 0.00082989
| Epoch 112 |   100/  632 batches | lr 0.00010 | ms/batch  9.33 | loss 0.00083884
| Epoch 112 |   125/  632 batches | lr 0.00010 | ms/batch  9.32 | loss 0.00089951
| Epoch 112 |   150/  632 batches | lr 0.00010 | ms/batch  7.88 | loss 0.00088895
| Epoch 112 |   175/  632 batches | lr 0.00010 | ms/batch  9.32 | loss 0.00086811
| Epoch 112 |   

| Epoch 115 |   525/  632 batches | lr 0.00010 | ms/batch  7.69 | loss 0.00083443
| Epoch 115 |   550/  632 batches | lr 0.00010 | ms/batch  9.33 | loss 0.00095853
| Epoch 115 |   575/  632 batches | lr 0.00010 | ms/batch  9.38 | loss 0.00086266
| Epoch 115 |   600/  632 batches | lr 0.00010 | ms/batch  9.36 | loss 0.00081949
| Epoch 115 |   625/  632 batches | lr 0.00010 | ms/batch  7.71 | loss 0.00082728
| Epoch 116 |    25/  632 batches | lr 0.00010 | ms/batch  8.10 | loss 0.00089549
| Epoch 116 |    50/  632 batches | lr 0.00010 | ms/batch  9.37 | loss 0.00084786
| Epoch 116 |    75/  632 batches | lr 0.00010 | ms/batch  7.85 | loss 0.00079398
| Epoch 116 |   100/  632 batches | lr 0.00010 | ms/batch  9.39 | loss 0.00080322
| Epoch 116 |   125/  632 batches | lr 0.00010 | ms/batch  9.54 | loss 0.00085743
| Epoch 116 |   150/  632 batches | lr 0.00010 | ms/batch  9.39 | loss 0.00084359
| Epoch 116 |   175/  632 batches | lr 0.00010 | ms/batch  7.86 | loss 0.00083860
| Epoch 116 |   

| Epoch 119 |   525/  632 batches | lr 0.00010 | ms/batch  9.38 | loss 0.00080400
| Epoch 119 |   550/  632 batches | lr 0.00010 | ms/batch  9.34 | loss 0.00093122
| Epoch 119 |   575/  632 batches | lr 0.00010 | ms/batch  7.72 | loss 0.00082710
| Epoch 119 |   600/  632 batches | lr 0.00010 | ms/batch  9.31 | loss 0.00078475
| Epoch 119 |   625/  632 batches | lr 0.00010 | ms/batch  9.37 | loss 0.00079437
| Epoch 120 |    25/  632 batches | lr 0.00010 | ms/batch  9.83 | loss 0.00086374
| Epoch 120 |    50/  632 batches | lr 0.00010 | ms/batch  7.84 | loss 0.00081831
| Epoch 120 |    75/  632 batches | lr 0.00010 | ms/batch 10.28 | loss 0.00076025
| Epoch 120 |   100/  632 batches | lr 0.00010 | ms/batch 10.08 | loss 0.00077076
| Epoch 120 |   125/  632 batches | lr 0.00010 | ms/batch  9.61 | loss 0.00082426
| Epoch 120 |   150/  632 batches | lr 0.00010 | ms/batch  7.79 | loss 0.00080822
| Epoch 120 |   175/  632 batches | lr 0.00010 | ms/batch  9.73 | loss 0.00081051
| Epoch 120 |   

| Epoch 123 |   525/  632 batches | lr 0.00010 | ms/batch  8.08 | loss 0.00073966
| Epoch 123 |   550/  632 batches | lr 0.00010 | ms/batch  9.86 | loss 0.00087868
| Epoch 123 |   575/  632 batches | lr 0.00010 | ms/batch  9.60 | loss 0.00081240
| Epoch 123 |   600/  632 batches | lr 0.00010 | ms/batch  9.89 | loss 0.00074685
| Epoch 123 |   625/  632 batches | lr 0.00010 | ms/batch  9.58 | loss 0.00076248
| Epoch 124 |    25/  632 batches | lr 0.00010 | ms/batch  8.69 | loss 0.00083765
| Epoch 124 |    50/  632 batches | lr 0.00010 | ms/batch  9.66 | loss 0.00077898
| Epoch 124 |    75/  632 batches | lr 0.00010 | ms/batch  9.94 | loss 0.00073020
| Epoch 124 |   100/  632 batches | lr 0.00010 | ms/batch  8.11 | loss 0.00073476
| Epoch 124 |   125/  632 batches | lr 0.00010 | ms/batch  9.62 | loss 0.00078848
| Epoch 124 |   150/  632 batches | lr 0.00010 | ms/batch  9.54 | loss 0.00081294
| Epoch 124 |   175/  632 batches | lr 0.00010 | ms/batch  9.60 | loss 0.00079539
| Epoch 124 |   

In [24]:
testing_results

,hidden_size,n_layers,act_fun,init_methods,mean_val_result,std_val_result,test_mse,test_mae,test_rmse,test_mape
0,400,8,LeakyReLU,xavier uniform,0.003321,0.000882,0.002097,0.033638,0.045793,-0.165342
1,400,4,ReLU,xavier uniform,0.003970,0.000540,0.002925,0.039573,0.054085,-0.053486
2,400,4,LeakyReLU,xavier uniform,0.003975,0.000493,0.003117,0.040846,0.055828,-0.000966
3,400,6,LeakyReLU,xavier uniform,0.003573,0.000707,0.002501,0.036594,0.050011,0.007922
4,600,4,LeakyReLU,xavier uniform,0.003256,0.000956,0.001807,0.031455,0.042512,0.104010


In [25]:
testing_results.to_csv('../results/testing_results_heston.csv')